In [25]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier


## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'mckayla-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to read from s3 bucket
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

##Reading the csv file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [26]:
## Removing the observations with N/As
heart = heart.dropna()

In [27]:
## Defining the input and target variables
X = heart[['male','age','currentSmoker','totChol','sysBP','BMI','heartRate', 'glucose']]
Y = heart['TenYearCHD']

## SPlitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [28]:
## Min- Max Transformation
scaler = MinMaxScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

In [29]:
## Identifying important variables with lasso
lasso_md = LassoCV().fit(X_train, Y_train)

## Extracting the best lambda
lasso_lambda = lasso_md.alpha_

##Fitting LASSO with best lambda
lasso_md = Lasso(alpha = lasso_lambda).fit(X_train, Y_train)
lasso_md.coef_

array([ 0.05681927,  0.25883001,  0.05106233,  0.04063307,  0.53065859,
        0.01984388, -0.00264956,  0.41360003])

In [30]:
## Defining the input and target variables for Logistic Regression
X_train_logit = X_train[['male','age','currentSmoker','totChol','sysBP', 'glucose']]
X_test_logit = X_test[['male','age','currentSmoker','totChol','sysBP', 'glucose']]

## Building the Logistic regression model
logit_md = LogisticRegression().fit(X_train_logit, Y_train)

## Predicting on test
logit_pred = logit_md.predict_proba(X_test_logit)[:, 1]

## Changing likelihoods to labels
logit_labels = np.where(logit_pred < 0.1, 0, 1)

## Computing the recall score
recall_score(Y_test, logit_labels)

0.9196428571428571

In [ ]:
## Random Forest Model

In [31]:
## Building the model
RF_md = RandomForestClassifier(n_estimators = 500).fit(X_train, Y_train)

## Extracting the feature importances
importances = pd.DataFrame({'Feature': X_train.columns, 'Importance': RF_md.feature_importances_})
importances = importances.sort_values(by = 'Importance', ascending = False)
importances

,Feature,Importance
4,sysBP,0.189025
5,BMI,0.172162
3,totChol,0.162119
7,glucose,0.159405
1,age,0.148250
6,heartRate,0.124008
0,male,0.023558
2,currentSmoker,0.021474


In [32]:
## Defining the input and target variables for Random Forest
X_train_RF = X_train[['sysBP','BMI','age','totChol', 'glucose']]
X_test_RF = X_test[['sysBP','BMI','age','totChol', 'glucose']]

## Building the Random Forest model
RF_md = RandomForestClassifier(n_estimators = 500).fit(X_train_RF, Y_train)

## Predicting on test
RF_pred = RF_md.predict_proba(X_test_RF)[:, 1]

## Changing likelihoods to labels
RF_labels = np.where(RF_pred < 0.1, 0, 1)

## Computing the recall score
recall_score(Y_test, RF_labels)

0.9464285714285714